In [2]:
from datetime import datetime
import warnings
import os
from collections import Counter

import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pymc as pm
import arviz as az
from scipy.stats import mode

from get_model_training_data_05 import get_features_and_data

%matplotlib inline
%config InlineBackend.figure_format = "retina"

sns.set(rc={"figure.figsize" : (25, 15)})
sns.set(font_scale=2)
sns.set_style("ticks")

warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [39]:
%load_ext autoreload
%autoreload 2

In [4]:
def precision_and_recall(df, true_col="release", pred_col="release_pred"):
    true_positives = len(df[(df[true_col] == 1) & (df[pred_col] == 1)])
    false_positives = len(df[(df[true_col] != 1) & (df[pred_col] == 1)])
    false_negatives = len(df[(df[true_col] == 1) & (df[pred_col] != 1)])

    return (np.round(true_positives / (true_positives + false_positives), 3),
            np.round(true_positives / (true_positives + false_negatives), 3))

## Load Training and Testing Data

In [111]:
(df, train_df, test_df, feature_names, next_two_weeks) = get_features_and_data()

training examples = 2500, testing examples = 325


In [112]:
feature_names

['days_since_previous_release',
 'month_holidays',
 'WD_Monday',
 'WD_Saturday',
 'WD_Sunday',
 'WD_Thursday',
 'WD_Tuesday',
 'WD_Wednesday',
 'previous_release',
 'previous_days_since_previous_release']

In [113]:
train_df["release"].value_counts()

release
0    2150
1     350
Name: count, dtype: Int64

In [114]:
train_df.groupby(["year", "release"]).size().reset_index().pivot(index="year", columns="release", values=0)

release,0,1
year,,
2017,270,39
2018,323,42
2019,312,53
2020,320,46
2021,310,55
2022,308,57
2023,307,58


## Model-Building and Evaluation

In [115]:
with pm.Model() as model:
    # data
    features = pm.MutableData("features", train_df[feature_names].T)
    # priors
    weights = pm.Normal("weights", mu=0, sigma=1, shape=len(feature_names))
    beta0 = pm.Normal("beta0", mu=0, sigma=1)
    # linear model
    release_prob = pm.math.invlogit(beta0 + pm.math.dot(weights, features))
    release = pm.Bernoulli(
        "release",
        p=release_prob,
        observed=train_df["release"].values,
        shape=features.shape[1],
    )

In [116]:
with model:
    idata = pm.sample(1000, tune=1000, chains=4, random_seed=1024)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [weights, beta0]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 57 seconds.


In [117]:
weights_df = az.summary(idata, round_to=2, var_names="weights")
weights_df["feature"] = feature_names
weights_df = weights_df.reset_index().drop(columns=["index"]).set_index("feature").sort_values(by=["mean"], ascending=False)
weights_df

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
feature,,,,,,,,,
previous_days_since_previous_release,0.82,0.58,-0.29,1.85,0.01,0.01,1597.17,1907.51,1.0
month_holidays,0.04,0.05,-0.06,0.13,0.00,0.00,3758.51,2503.46,1.0
WD_Thursday,-0.10,0.19,-0.46,0.24,0.00,0.00,2697.63,3005.24,1.0
WD_Wednesday,-0.39,0.19,-0.77,-0.06,0.00,0.00,2628.77,2853.31,1.0
WD_Saturday,-0.55,0.21,-0.96,-0.17,0.00,0.00,2847.25,2532.33,1.0
days_since_previous_release,-0.63,0.58,-1.65,0.48,0.01,0.01,1597.14,1869.76,1.0
WD_Tuesday,-0.93,0.21,-1.30,-0.53,0.00,0.00,2786.32,2940.91,1.0
WD_Sunday,-1.20,0.23,-1.64,-0.76,0.00,0.00,2926.73,2944.18,1.0
previous_release,-1.61,0.38,-2.36,-0.93,0.01,0.00,4341.04,2689.92,1.0


### Out-of-Sample Evaluation

In [118]:
with model:
    pm.set_data({"features" : test_df[feature_names].T})
    pred_test = pm.sample_posterior_predictive(idata, predictions=True, var_names=["release"])

Sampling: [release]


Output()

In [119]:
test_df["release_pred"] = [mode(p)[0] for p in pred_test["predictions"]["release"].stack(all_draws=["chain", "draw"]).values]

In [120]:
test_df["release_pred"].value_counts()

release_pred
0    325
Name: count, dtype: int64

In [121]:
test_df["release"].value_counts()

release
0    269
1     56
Name: count, dtype: Int64

In [106]:
precision_and_recall(test_df)

ZeroDivisionError: division by zero

In [82]:
test_df["release_pred_prob"] = np.mean(pred_test["predictions"]["release"].stack(all_draws=["chain", "draw"]).values, axis=1)

In [66]:
ax = sns.histplot(x=test_df["prob_of_release"], alpha=0.8, label="Actual Test Set Distribution")
sns.histplot(x=test_df["release_pred_prob"], ax=ax, label="Predicted Test Set Distribution")
ax.legend()
plt.show()

KeyError: 'prob_of_release'